# Surprise 를 이용한 개인화 영화 추천 시스템 구축
- SVD(Singular Value Decomposition) 활용

In [1]:
import pandas as pd

In [14]:
ratings = pd.read_csv('data/ml-latest/ratings_noh.csv', header=None)
ratings.columns = 'userId movieId rating timestamp'.split()
ratings.head(3)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224


In [11]:
from surprise import SVD, Reader
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성. 
data_folds = DatasetAutoFolds(ratings_file='data/ml-latest/ratings_noh.csv', reader=reader)

#전체 데이터를 학습데이터로 생성함. 
trainset = data_folds.build_full_trainset()

In [12]:
algo = SVD(n_epochs=20, n_factors=50, random_state=2022)
algo.fit(trainset)

- 사용자 아이디: 9, 영화 아이디: 42 (Dead Presidents)

In [15]:
# 영화에 대한 상세 속성 정보 DataFrame로딩
movies = pd.read_csv('data/ml-latest/movies.csv')

# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인. 
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [16]:
uid = str(9)
mid = str(42)

pred = algo.predict(uid, mid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.25   {'was_impossible': False}


- 사용자 아이디 9번이 보지 않은 영화 추천

In [18]:
def get_unseen_surprise(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()
    
    # 모든 영화들의 movieId를 리스트로 생성. 
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print(f'평점 매긴 영화수: {len(seen_movies)}, 추천대상 영화수: {len(unseen_movies)}, 전체 영화수: {len(total_movies)}')
    
    return unseen_movies

In [19]:
unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수: 46, 추천대상 영화수: 9696, 전체 영화수: 9742


In [21]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [int(pred.iid) for pred in top_predictions]
    top_movie_rating = [pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [(id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

In [22]:
unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)
print('===== Top-10 추천 영화 리스트 =====')
for movie in top_movie_preds:
    print(movie[1], ":", movie[2])

평점 매긴 영화수: 46, 추천대상 영화수: 9696, 전체 영화수: 9742
===== Top-10 추천 영화 리스트 =====
Usual Suspects, The (1995) : 4.070330794979969
Shawshank Redemption, The (1994) : 4.063731956995097
Philadelphia Story, The (1940) : 4.051908410348554
Lawrence of Arabia (1962) : 4.0227662213503805
Ran (1985) : 4.011500870494226
Evil Dead II (Dead by Dawn) (1987) : 3.9996958730949137
Boogie Nights (1997) : 3.9853484799628194
Boondock Saints, The (2000) : 3.9798846903676015
Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001) : 3.9786251266485744
Spotlight (2015) : 3.978415496006661
